In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/muheshaam/arabic")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/muheshaam/arabic


100%|██████████| 4.22M/4.22M [00:01<00:00, 3.37MB/s]


In [3]:
!pip install emoji
!pip install arabic-reshaper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 9.9 MB/s eta 0:00:00


In [7]:
!pip install python-bidi

In [8]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
import re
import string
import emoji
import random
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# reading the train data
df_train =pd.read_csv("/content/arabic/train.csv")

In [10]:
# print sample of the data
df_train.head()

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...


In [11]:
# reading the vaild data
df_valid=pd.read_csv("/content/arabic/valid.csv")

In [12]:
# print a sample of valid data
df_valid.head(10)

,correct,incorrect
0,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...
1,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...
2,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...
3,علي من ميلانو يقول : ان سوريا بغالبيتها التي ت...,علي من ميلانوا يقول ان سوريا بغالبيته التي تبل...
4,قسما بالله ، انا اعرف اهل حمص ، اليوم شيعنا خي...,قسما بالله انا اعرف اهل حمص اليوم شيعنا خيره ا...
5,اصلا لا توجد القاعده في بلاد المغرب العربي ، ب...,اصلا لاتوجد القاعده في بلاد المغرب العربي بل ا...
6,بصراحه ، عندما كانت الجزيره جزيره العرب كنا نش...,بصراحه عندما كانت الجزيره جزيره العرب كنا نشاه...
7,اذا حزب الجنوب اللبناني اعلن الحرب علي الشعب ا...,اذا حزب الجنوب اللبناني اعلن الحرب علي الشعب ا...
8,ان هءلاء القتلي هم شباب قتلهم الخوارج ، ولمن ي...,ان هاءلاء القتلي هم شباب قتلهم الخوارج و لمن ي...
9,بعض الذين يدعمون الارهاب يحاولون تبرءه القتله ...,بعض الذين يدعمون الارهاب يحاولون تبرءه القتله ...


In [13]:
df = pd.concat([df_train, df_valid])
df

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...,( . . وهو يصلح جدران بيته ويصبغها وحينما يستعص...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20643 entries, 0 to 2292
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   correct    20643 non-null  object
 1   incorrect  20643 non-null  object
dtypes: object(2)
memory usage: 483.8+ KB


In [15]:
stop = set(nltk.corpus.stopwords.words("arabic"))

# Text length analysis
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

# Counting emoji
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

df['word_count_correct'] = df['correct'].apply(lambda x: len(str(x).split(" ")))
df['word_count_incorrect'] = df['incorrect'].apply(lambda x: len(str(x).split(" ")))
df['stopwords_correct'] = df['correct'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['stopwords_incorrect'] = df['incorrect'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['emoji_count_correct'] = df['correct'].apply(lambda x: emoji_counter(x))
df['emoji_count_incorrect'] = df['incorrect'].apply(lambda x: emoji_counter(x))
df.head()

,correct,incorrect,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...,65,59,12,8,0,0
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...,54,45,10,10,0,0
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,66,57,7,7,0,0
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...,70,62,10,10,0,0
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,66,64,11,11,0,0


In [16]:
df.describe()

,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect
count,20643.000000,20643.000000,20643.00000,20643.000000,20643.0,20643.0
mean,58.711718,54.240566,9.33561,9.744853,0.0,0.0
std,18.513807,17.561493,4.53619,5.036631,0.0,0.0
min,6.000000,6.000000,0.00000,0.000000,0.0,0.0
25%,51.000000,47.000000,7.00000,7.000000,0.0,0.0
50%,57.000000,52.000000,9.00000,9.000000,0.0,0.0
75%,63.000000,58.000000,11.00000,12.000000,0.0,0.0
max,484.000000,461.000000,132.00000,132.000000,0.0,0.0


In [17]:
# Function to add a random character to a word in Arabic text
def add_random_arabic_character(word):
    arabic_chars = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"
    random_char = random.choice(arabic_chars)
    return word + random_char

# Function to remove a random character from a word in Arabic text
def remove_random_arabic_character(word):
    if len(word) <= 1:
        return word  # If the word has only one character, return the same word
    random_index = random.randint(0, len(word) - 1)  # Choose a random index
    return word[:random_index] + word[random_index + 1:]  # Remove the character at the random index

In [18]:
def modify_text(text):
    words = text.split()  # Split the text into words
    if not words:
        return ""  # Return empty string if there are no words
    random_word = random.choice(words)  # Choose a random word
    modify_choice = random.choice(["add", "remove"])  # Choose whether to add or remove a character
    modified_words = []
    for word in words:
        if word == random_word:
            if modify_choice == "add":
                modified_words.append(add_random_arabic_character(word))
            elif modify_choice == "remove":
                modified_words.append(remove_random_arabic_character(word))
        else:
            modified_words.append(word)
    return ' '.join(modified_words)  # Join the modified words back into a single string

In [19]:
df['new_incorrect'] = df['correct'].apply(modify_text)

df.head(10)

,correct,incorrect,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect,new_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...,65,59,12,8,0,0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...,54,45,10,10,0,0,النصر ات لا محال ان شاء الله . من يءمن بالله و...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,66,57,7,7,0,0,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...,70,62,10,10,0,0,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,66,64,11,11,0,0,النظامع الاسدي تجاوز حتي ما فعله معمر القذافي ...
5,سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنته...,سوريا اما للسوريين او لبشار وعصابته لن تنتهي ه...,62,52,8,8,0,0,سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنته...
6,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون وي...,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون و ...,50,49,4,9,0,0,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون وي...
7,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا ، ن...,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا نحن...,53,52,10,14,0,0,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا ، ن...
8,تنويه : عقرب هي قريه تقع في سهل الحوله ، وتتبع...,تنويه : عقرب هي قريه تقع في سهل الحوله وتتبع ا...,49,45,7,9,0,0,تنويه : عقرب هي قريه تقع في سهل الحوله ، وتتبع...
9,ضحكوا علينا بحرب العراق عام 1990 ، وعام 2003 ،...,ضحكو علينا بحرب العراق عام ١٩٩٠ وعام ٢٠٠٣ وحرب...,56,50,1,2,0,0,ضحكوا علينا بحرب العراق عام 1990 ، وعام 2003 ،...


In [20]:
df=df[["correct","new_incorrect"]]
df

,correct,new_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال ان شاء الله . من يءمن بالله و...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظامع الاسدي تجاوز حتي ما فعله معمر القذافي ...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...
2289,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...,( . . وهو يصلحظ جدران بيته ويصبغها ، وحينما يس...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...


In [21]:
arabicPunctuations = [".","#","$","//","?","=","'","_","-","'",";","\\","`","؛","<",">","(",")","*","&","^","%","]","[",",","ـ","،","/",":","؟",".","'","{","}","~","|","!","”","…","“","–"] # defining customized punctuation marks
englishPunctuations = string.punctuation # importing English punctuation marks
englishPunctuations = [word.strip() for word in englishPunctuations] # converting the English punctuation from a string to array for processing
punctuationsList = arabicPunctuations + englishPunctuations # creating a list of all punctuation marks

def removingPunctuation(text):
  cleanTweet = ''
  for i in text:
    if i not in arabicPunctuations:
      cleanTweet = cleanTweet + '' + i
  return cleanTweet

df['correct'] = df['correct'].apply(lambda x: removingPunctuation(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x: removingPunctuation(x))


df.sample(5)

,correct,new_incorrect
6796,اقسم بالله يا اخواني اننا راينا باعيننا الميلي...,اقسم بالله يا اخواني اننا راينا باعيننا الميلي...
5637,لم يرتكب صدام ولا القذافي حتي اعدامهما واحدا...,لم يرتكب صدام ولا القذافي حتي اعدامهما واحدا...
11542,ليس لهذه الزياره اي معني طالما مازالت العراق ت...,ليس لهذه الزياره اي معني طالما مازالت العراق ت...
407,انا مع الاخ ياسر الزغبي تغطيه الجزيره للاحداث...,انا مع الاخ ياسر الزغبي تغطيه الجزيره للاحداث...
5262,انهاء العنف لن يكون الا بطريقه واحده وهي انسح...,انهاء العنف لن يكون الا بطريقه وحده وهي انسحا...


In [22]:
def clean(text):
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    # Remove numbers
    text = re.sub("\d+", " ", text)
    return text

df['correct'] = df['correct'].apply(lambda x:clean(x))
df['correct'] = df['correct'].apply(lambda x:clean(x))

df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))

In [23]:
df

,correct,new_incorrect
0,سبحان الله الحكام العرب سيموت علي الكرسي ليظهر...,سبحان الله الحكام العرب سيموت علي الكرسي ليظهر...
1,النصر ات لا محال ان شاء الله من يءمن بالله وال...,النصر ات لا محال ان شاء الله من يءمن بالله وال...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته لحد الان غير مستوعبين انه رح ينق...,الاسد وعصابته لحد الان غير مستوعبين انه رح ينق...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظامع الاسدي تجاوز حتي ما فعله معمر القذافي ...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,وهو يصلح جدران بيته ويصبغها وحينما يستعصي علي...,وهو يصلحظ جدران بيته ويصبغها وحينما يستعصي عل...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...


In [24]:
df.sample(10)

,correct,new_incorrect
5303,واشار الي ان الميزانيه السعوديه من المتوقع ان...,واشار الي ان الميزانيه السعوديه من المتوقع ان...
17056,كل هذا كلام غير نافع نسبه المسلمين في امريكا ل...,كل هذا كلام غير نافع نسبه المسلمين في امريكا ل...
17952,لا يستطيع احد ان ينكر ان القاعده صناعه يمنيه ب...,لا يستطيع احد ان ينكر ان القاعده صناعه يمنيه ب...
1877,لا شيء جديد سوي ان مرسيل داءما لديه الجديد وهو...,لا شيء جديد سوي ان مرسيل داءما لديه الجديد وهو...
15986,هذه مءذنه مسجد ويمكن انقصفت بالغلط والا الغلط ...,هذه مءذنه مسجد ويمكن اقصفت بالغلط والا الغلط ب...
1980,الاخوان المسلمون لا يمثلون شيءا في الشارع الار...,الاخوان المسلمون لا يمثلون شيءا في الشارع الار...
3005,كلام خالد رقم مضحك عندما يقول ان السعوديه قادر...,كلام خالد رقم مضحك عندما يقول ان السعوديه قادر...
97,احب ان اوجه كلامي الي كل من ينتقد حماس اتقوا ا...,احب ان اوجه كلامي الي كل ن ينتقد حماس اتقوا ال...
1124,لم اكن اتوقع حدوث ذلك لكنه حدث الناس بدات تكره...,لم اكن اتوقع حدوث ذلك لكنه حدث الناس بدات تكره...
13806,فعلا زمان قبل الستلايت كنا ليلا نهارا نسمع اغا...,فعلا زان قبل الستلايت كنا ليلا نهارا نسمع اغان...


In [25]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(df['correct'])

In [26]:
correct_sequences = tokenizer.texts_to_sequences(df['correct'])
incorrect_sequences = tokenizer.texts_to_sequences(df['new_incorrect'])

In [27]:
# Set the maximum sequence length

max_sequence_length = max(max(len(seq) for seq in correct_sequences), max(len(seq) for seq in incorrect_sequences))

# Pad sequences to ensure uniform length
correct_sequences = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post')
incorrect_sequences = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post')

In [28]:
len(tokenizer.word_index)

100874

In [29]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(incorrect_sequences, correct_sequences, test_size=0.2, random_state=42)

In [30]:
# Model architecture
input_seq = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_seq)
encoder = LSTM(128, return_sequences=True)(embedding_layer)
decoder = Dense(len(tokenizer.word_index) + 1, activation='softmax')(encoder)

# Define the model
model = Model(input_seq, decoder)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 394)]             0         
                                                                 
 embedding (Embedding)       (None, 394, 128)          12912000  
                                                                 
 lstm (LSTM)                 (None, 394, 128)          131584    
                                                                 
 dense (Dense)               (None, 394, 100875)       13012875  
                                                                 
Total params: 26056459 (99.40 MB)
Trainable params: 26056459 (99.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=10)

Epoch 1/10
1033/1033 [==============================] - 194s 185ms/step - loss: 1.4794 - accuracy: 0.8756 - val_loss: 1.0611 - val_accuracy: 0.8827
Epoch 2/10
1033/1033 [==============================] - 176s 171ms/step - loss: 0.9623 - accuracy: 0.8951 - val_loss: 0.8996 - val_accuracy: 0.9057
Epoch 3/10
1033/1033 [==============================] - 176s 170ms/step - loss: 0.7881 - accuracy: 0.9132 - val_loss: 0.7814 - val_accuracy: 0.9177
Epoch 4/10
1033/1033 [==============================] - 175s 169ms/step - loss: 0.6416 - accuracy: 0.9258 - val_loss: 0.7058 - val_accuracy: 0.9246
Epoch 5/10
1033/1033 [==============================] - 175s 169ms/step - loss: 0.5247 - accuracy: 0.9369 - val_loss: 0.6566 - val_accuracy: 0.9284
Epoch 6/10
1033/1033 [==============================] - 175s 170ms/step - loss: 0.4327 - accuracy: 0.9458 - val_loss: 0.6288 - val_accuracy: 0.9302
Epoch 7/10
1033/1033 [==============================] - 174s 169ms/step - loss: 0.3601 - accuracy: 0.9530 - val_

In [33]:
# Save the model
model.save('auto_correction_model.h5')

In [34]:
# Tokenize the input text
input_text =str(input())
input_sequence = tokenizer.texts_to_sequences([input_text])

# Pad the input sequence
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

# Predict the corrected output
output_sequence = model.predict(padded_input_sequence)
corrected_output_sequence = np.argmax(output_sequence, axis=-1)

# Convert the corrected output sequence back to text
corrected_output_text = tokenizer.sequences_to_texts(corrected_output_sequence)

print("Corrected Output:", corrected_output_text)

يجب قول الح
1/1 [==============================] - 0s 387ms/step
Corrected Output: ['يجب قول']


In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
# Assuming you already have a tokenizer object named 'tokenizer'

# Save the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

In [37]:
def predict(input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])

    # Pad the input sequence
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

    # Predict the corrected output
    output_sequence = model.predict(padded_input_sequence)
    corrected_output_sequence = np.argmax(output_sequence, axis=-1)

    # Convert the corrected output sequence back to text
    corrected_output_text = tokenizer.sequences_to_texts(corrected_output_sequence)

    return corrected_output_text

In [38]:
sentences = [
    "علي العب العربي التركيز",
    "يجب قول الحف",
    "الحياه ف سوريا صعبه",
    "سلام علي الوطن العبي",
    "اذا اردت ان تضحك ليك بمشاهده برامج التلفزيون",
    "الذهاب و العوده انر صعب",
    "اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء",
]
len (sentences)

7

In [39]:
prediction=[]
for i in range(7):
    corrected_text = predict(sentences[i])
    print("Corrected text:", corrected_text)
    prediction.append(corrected_text)

1/1 [==============================] - 0s 26ms/step
Corrected text: ['علي الشعب العربي التركيز']
1/1 [==============================] - 0s 25ms/step
Corrected text: ['يجب قول']
1/1 [==============================] - 0s 25ms/step
Corrected text: ['الحياه في سوريا صعبه']
1/1 [==============================] - 0s 24ms/step
Corrected text: ['سلام علي الوطن العربي']
1/1 [==============================] - 0s 25ms/step
Corrected text: ['اذا اردت ان تضحك بمشاهده برامج التلفزيون']
1/1 [==============================] - 0s 26ms/step
Corrected text: ['الذهاب و العوده صعب']
1/1 [==============================] - 0s 26ms/step
Corrected text: ['اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء']
